# Cálculo de Energia Livre em Catálise usando VASP e ASE

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

### Carregando parâmetros

In [23]:
import os
# Definindo o path para os arquivos de potencial de pseudopotenciais do VASP
# Certifique-se de que o caminho esteja correto para o seu sistema
os.environ['VASP_PP_PATH'] = '/home/marvin/Programs/vasp-6.5.1/pp'
os.environ['ASE_VASP_COMMAND'] = 'mpirun -np 1 vasp_std_gpu'
os.environ['NO_STOP_MESSAGE'] = '1' # to avoid warning from mpirun

# Importando o VASP calculator do ASE
from ase.calculators.vasp import Vasp

from ase import Atoms
from ase.io import write, read
from ase.visualize import view

import numpy as np

In [24]:
# ==========================================================
# ISOMERIZAÇÃO cis-COOH → trans-COOH sobre Pt(111)
# Adaptado de Gokhale et al. (JACS 2008)
# ==========================================================

from ase import Atoms
from ase.build import fcc111, add_adsorbate
from ase.constraints import FixAtoms
from ase.lattice.cubic import FaceCenteredCubic
from ase.calculators.vasp import Vasp
from matplotlib import pyplot as plt
import numpy as np

# ----------------------------------------------------------
# Parâmetros gerais
# ----------------------------------------------------------
T = 500.0       # Temperatura (K)
P = 101325.0
encut = 450
kpts_bulk = [6,6,6]
kpts_slab = [6,6,1]


### Bulk e superfície de Pt

In [25]:
# ==========================================================
# Bulk de Pt
# ==========================================================
Pt_crystal = FaceCenteredCubic(size=(1,1,1), symbol='Pt', latticeconstant=3.967, pbc=True)

calc_bulk = Vasp(directory='Pt_bulk_relax',
                 xc='PBE', encut=encut, kpts=kpts_bulk,
                 ismear=0, sigma=0.05,
                 ediff=1e-6, ediffg=-0.01,
                 isif=7, ibrion=2, nsw=100,
                 atoms=Pt_crystal)
calc_bulk.calculate(Pt_crystal)

E_Pt = Pt_crystal.get_potential_energy()
a0_Pt = Pt_crystal.cell.cellpar()[0]
print(f"E(Pt bulk) = {E_Pt:.3f} eV")
print(f"Lattice constant (a₀) = {a0_Pt:.3f} Å")

E(Pt bulk) = -24.458 eV
Lattice constant (a₀) = 3.967 Å


#### Superfície limpa de Pt(111)

In [26]:
# ==========================================================
# Superfície limpa de Pt(111)
# ==========================================================
Pt111 = fcc111("Pt", size=(2,2,3), a=a0_Pt)
Pt111.center(vacuum=5.0, axis=2)
Pt111.pbc = True

calc_clean = Vasp(directory='Pt111_clean_relax',
                  xc='PBE', encut=encut, kpts=kpts_slab,
                  ismear=0, sigma=0.05,
                  ediff=1e-6, ediffg=-0.01,
                  isif=2, ibrion=2, nsw=80,
                  atoms=Pt111)
calc_clean.calculate(Pt111)

E_Pt111 = Pt111.get_potential_energy()
print(f"E(Pt111 clean) = {E_Pt111:.3f} eV")

E(Pt111 clean) = -67.930 eV


### Otimização das espécies gasosas

In [27]:
# ==========================================================
# Otimização do cis-COOH
# ==========================================================
cis_cooh_relax = Atoms('COOH',
                 positions=[[0.00,  0.00, 0.00],   # C
                            [1.20,  0.00, 0.00],   # O carbonílico
                            [-0.60, 0.85, 0.00],   # O hidroxílico
                            [-0.55, -0.55, 0.75]]) # H voltado para fora
cis_cooh_relax.center(vacuum=5.0)  
cis_cooh_relax.pbc = True

calc_relax = Vasp(
    directory='cis_cooh_relax',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-6, ediffg=-0.01,
    isif=2, ibrion=2, nsw=100,
    nelm=100,
    lwave=True, lcharg=True, lvtot=True,
    atoms=cis_cooh_relax
)
calc_relax.calculate(cis_cooh_relax) # demora por volta de 10 s

E_cis_cooh_relax = cis_cooh_relax.get_potential_energy()

print(f'Optimized energy of CO2: {E_cis_cooh_relax:.3f} eV')

Optimized energy of CO2: -24.058 eV


In [45]:
# ==========================================================
# Otimização do trans-COOH
# ==========================================================
cooh_relax = Atoms('COOH',
             positions=[
                 [  0.00,  0.00,  0.00],   # C central
                 [ -1.25, -0.35, -0.25],   # O carbonílico (ligado ao Pt, off-top)
                 [  1.15,  0.85,  0.15],   # O hidroxílico
                 [  1.55,  0.65, -0.75]    # H do grupo OH voltado para a superfície
             ])

cooh_relax.center(vacuum=5.0)  
cooh_relax.pbc = True

calc_relax = Vasp(
    directory='cooh_relax',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-6, ediffg=-0.01,
    isif=2, ibrion=2, nsw=100,
    nelm=100,
    lwave=True, lcharg=True, lvtot=True,
    atoms=cooh_relax
)
calc_relax.calculate(cooh_relax) # demora por volta de 10 s

E_cooh_relax = cooh_relax.get_potential_energy()

print(f'Optimized energy of CO2: {E_cooh_relax:.3f} eV')

Optimized energy of CO2: -24.054 eV


### cis-COOH(ads) e trans-COOH(ads) sobre Pt(111)

In [ ]:
# ==========================================================
# cis-COOH(ads) sobre Pt(111)
# ==========================================================
cis_cooh = Atoms('COOH',
                 positions=[[0.00,  0.00, 0.00],   # C
                            [1.20,  0.00, 0.00],   # O carbonílico
                            [-0.60, 0.85, 0.00],   # O hidroxílico
                            [-0.55, -0.55, 0.75]]) # H voltado para fora

cis_slab = fcc111("Pt", size=(2,2,3), a=a0_Pt)
cis_slab.center(vacuum=5.0, axis=2)
cis_slab.pbc = True
add_adsorbate(cis_slab, adsorbate=cis_cooh, height=2.0, position=(4.6,2.2))

constraint = FixAtoms(mask=[atom.symbol=='Pt' for atom in cis_slab])
cis_slab.set_constraint([constraint])

calc_cis = Vasp(directory='Pt111_cisCOOH_relax',
                xc='PBE', encut=encut, kpts=kpts_slab,
                ismear=0, sigma=0.05,
                ediff=1e-6, ediffg=-0.02,
                isif=0, ibrion=2, nsw=120,
                atoms=cis_slab)
calc_cis.calculate(cis_slab)
E_cisCOOH_ads = cis_slab.get_potential_energy()
print(f"E(Pt111 + cis-COOH) = {E_cisCOOH_ads:.3f} eV")

# Demorou 936m e 57s

E(Pt111 + cis-COOH) = -94.323 eV


In [46]:
# ==========================================================
# COOH(ads) (trans ou “normal”) sobre Pt(111)
# ==========================================================
cooh = Atoms('COOH',
             positions=[
                 [  0.00,  0.00,  0.00],   # C central
                 [ -1.25, -0.35, -0.25],   # O carbonílico (ligado ao Pt, off-top)
                 [  1.15,  0.85,  0.15],   # O hidroxílico
                 [  1.55,  0.65, -0.75]    # H do grupo OH voltado para a superfície
             ])

cooh_slab = fcc111("Pt", size=(2,2,3), a=a0_Pt)
cooh_slab.center(vacuum=5.0, axis=2)
cooh_slab.pbc = True
add_adsorbate(cooh_slab, adsorbate=cooh, height=2.0, position=(4.6,2.1))
cooh_slab.set_constraint([constraint])

calc_cooh = Vasp(directory='Pt111_COOH_relax',
                 xc='PBE', encut=encut, kpts=kpts_slab,
                 ismear=0, sigma=0.05,
                 ediff=1e-6, ediffg=-0.02,
                 isif=0, ibrion=2, nsw=120,
                 atoms=cooh_slab)
calc_cooh.calculate(cooh_slab)
E_COOH_ads = cooh_slab.get_potential_energy()
print(f"E(Pt111 + COOH) = {E_COOH_ads:.3f} eV")

# Demorou 1078m e 17.6s

E(Pt111 + COOH) = -94.666 eV


### Cálculo da energia

In [47]:
# ==========================================================
# 5. ΔE (cis→trans)
# ==========================================================
ΔE_cis = E_cisCOOH_ads - E_cis_cooh_relax - E_Pt111
ΔE_trans = E_COOH_ads - E_cooh_relax - E_Pt111
print(f"ΔE (cis→COOH) = {ΔE_cis:.3f} eV")
print(f"ΔE (cis→COOH) = {ΔE_trans:.3f} eV")

ΔE (cis→COOH) = -2.335 eV
ΔE (cis→COOH) = -2.682 eV


### Entalpia

In [48]:
# ==========================================================
# Entalpia do cis_COOH
# ==========================================================

calc_vib = Vasp(
    directory='cis_COOH_vib',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-8,
    ibrion=6,           
    lreal=False,
    lwave=False, lcharg=False,
    atoms=cis_cooh_relax
)

calc_vib.calculate(cis_cooh_relax)

vib_COOH_cis_gas = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = vib_COOH_cis_gas.get_frequencies()

print(f'Vibrational frequencies of COOH Cis gas (cm⁻¹):')
vib_energies_COOH = []
for freq in freqs:
    if freq.real > 100:                                 # TROCAMOS
        vib_energy = (freq.real * 1.23981e-4)  
        vib_energies_COOH.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

# Demorou 3m e 32.9s


Vibrational frequencies of COOH Cis gas (cm⁻¹):
f_v: 547.539 cm⁻¹, E_v: 0.068 eV
f_v: 590.782 cm⁻¹, E_v: 0.073 eV
f_v: 1038.902 cm⁻¹, E_v: 0.129 eV
f_v: 1180.074 cm⁻¹, E_v: 0.146 eV
f_v: 1832.969 cm⁻¹, E_v: 0.227 eV
f_v: 3697.924 cm⁻¹, E_v: 0.458 eV


In [49]:
from ase.thermochemistry import IdealGasThermo

COOH_thermo_cis_gas = IdealGasThermo(vib_energies=vib_energies_COOH,
                        potentialenergy=E_cis_cooh_relax, atoms=cis_cooh_relax,
                        geometry='nonlinear', symmetrynumber=2, spin=0)

G_COOH_cis_gas = COOH_thermo_cis_gas.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_COOH_cis_gas = COOH_thermo_cis_gas.get_enthalpy(temperature=T, verbose=False)

print(f'ΔH COOH cis gas = {H_COOH_cis_gas:.3f} eV')

ΔH COOH cis gas = -23.287 eV


In [50]:
# ==========================================================
# Entalpia do trans_COOH
# ==========================================================

calc_vib = Vasp(
    directory='trans_COOH_vib',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-8,
    ibrion=6,           
    lreal=False,
    lwave=False, lcharg=False,
    atoms=cooh_relax
)

calc_vib.calculate(cooh_relax)

vib_COOH_trans_gas = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = vib_COOH_trans_gas.get_frequencies()

print(f'Vibrational frequencies of COOH Trans gas (cm⁻¹):')
vib_energies_COOH_trans = []
for freq in freqs:
    if freq.real > 100:                                 # TROCAMOS
        vib_energy = (freq.real * 1.23981e-4)  
        vib_energies_COOH_trans.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

# Demorou 3m e 46.7s

Vibrational frequencies of COOH Trans gas (cm⁻¹):
f_v: 539.326 cm⁻¹, E_v: 0.067 eV
f_v: 589.957 cm⁻¹, E_v: 0.073 eV
f_v: 1038.271 cm⁻¹, E_v: 0.129 eV
f_v: 1179.650 cm⁻¹, E_v: 0.146 eV
f_v: 1833.661 cm⁻¹, E_v: 0.227 eV
f_v: 3701.173 cm⁻¹, E_v: 0.459 eV


In [51]:
from ase.thermochemistry import IdealGasThermo

COOH_thermo_trans_gas = IdealGasThermo(vib_energies=vib_energies_COOH,
                        potentialenergy=E_cooh_relax, atoms=cooh_relax,
                        geometry='nonlinear', symmetrynumber=2, spin=0)

G_COOH_trans_gas = COOH_thermo_trans_gas.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_COOH_trans_gas = COOH_thermo_trans_gas.get_enthalpy(temperature=T, verbose=False)

print(f'ΔH COOH cis gas = {H_COOH_trans_gas:.3f} eV')

ΔH COOH cis gas = -23.283 eV


In [52]:
# ==========================================================
# Entalpia espécies adsorvidas
# ==========================================================
from ase.thermochemistry import IdealGasThermo
import time

def calc_vibrational_energy(structure, directory):
    calc_vib = Vasp(directory=directory,
                    xc='PBE', kpts=kpts_slab, encut=encut,
                    ismear=0, sigma=0.05,
                    ediff=1e-8, ibrion=5, nfree=2, nsw=1,
                    lreal=False, lwave=False, lcharg=False,
                    atoms=structure)
    calc_vib.calculate(structure)
    vib = calc_vib.get_vibrations()
    freqs = vib.get_frequencies()
    vib_energies = [(f.real * 1.23981e-4) for f in freqs if f.real > 150]
    return vib_energies

t1 = time.time()
print('========== vibração cis ads ==========')
vibE_cis  = calc_vibrational_energy(cis_slab,  'Pt111_cisCOOH_vib')
t2 = time.time()
print('Tempo de execução: ', (t2-t1)/3600, 'horas')
print('=========== Fim ==========')

t1 = time.time()
print('========== vibração trans ads ==========')
vibE_cooh_trans = calc_vibrational_energy(cooh_slab, 'Pt111_COOH_vib')
t2 = time.time()
print('Tempo de execução: ', (t2-t1)/3600, 'horas')
print('=========== Fim ==========')

thermo_cis = IdealGasThermo(vib_energies=vibE_cis, potentialenergy=E_cisCOOH_ads,
                            atoms=cis_slab, geometry='nonlinear', symmetrynumber=1, spin=0)
thermo_cooh = IdealGasThermo(vib_energies=vibE_cooh_trans, potentialenergy=E_COOH_ads,
                             atoms=cooh_slab, geometry='nonlinear', symmetrynumber=1, spin=0)

H_cis  = thermo_cis.get_enthalpy(temperature=T, verbose=False)
H_cooh_trans = thermo_cooh.get_enthalpy(temperature=T, verbose=False)

ΔH_cis = H_cis - H_COOH_cis_gas - E_Pt111
ΔH_trans = H_cooh_trans - H_COOH_cis_gas - E_Pt111
print(f"ΔH (cis→COOH) = {ΔH_cis:.3f} eV")
print(f"ΔH (trans→COOH) = {ΔH_trans:.3f} eV")


========== vibração cis ads ==========
Tempo de execução:  5.175953625506825 horas
=========== Fim ==========
========== vibração trans ads ==========
Tempo de execução:  5.557619536585278 horas
=========== Fim ==========
ΔH (cis→COOH) = -2.216 eV
ΔH (trans→COOH) = -2.557 eV
